In [ ]:
#hide
#default_exp utils.resume_from_checkpoint
from nbdev.showdoc import *
from block_types.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Resume utilities

> Routines for resuming from previous experiments

In [ ]:
#export
import pickle
import os
import numpy as np

import hpsearch.config.hp_defaults as dflt

## exists_current_checkpoint

In [ ]:
#export
def exists_current_checkpoint (parameters, path_results):

    model_extension = parameters.get('model_extension', 'h5')

    return os.path.exists('%s/best_model.%s' %(path_results, model_extension))

## obtain_last_result

In [ ]:
#export    
def obtain_last_result (parameters, path_results):
    
    if parameters.get('use_last_result_from_dict', False):
        return obtain_last_result_from_dict (parameters, path_results)
    if 'result_file' in parameters.keys():
        name_result_file = parameters['result_file']
    else:
        name_result_file = parameters.get('name_model_history', 'model_history.pk')
    path_results_file = '%s/%s' %(path_results, name_result_file)
    dict_results = None
    if os.path.exists (path_results_file):
        history = pickle.load(open(path_results_file, 'rb'))
        metrics = parameters.get('key_scores')
        if metrics is None:
            metrics = history.keys()
        ops = parameters.get('ops')
        if ops is None:
            ops = ['max'] * len(metrics)
        if type(ops) is str:
            ops = [ops] * len(metrics)
        if type(ops) is dict:
            ops_dict = ops
            ops = ['max'] * len(metrics)
            i = 0
            for k in metrics:
                if k in ops_dict.keys():
                    ops[i] = ops_dict[k]
                i += 1
        dict_results = {}
        max_last_position = -1
        for metric, op in zip(metrics, ops):
            if metric in history.keys():
                history_array = history[metric]
                score = min(history_array) if op == 'min' else max(history_array)
                last_position = np.where(np.array(history_array).ravel()==0)[0]
                if len(last_position) > 0:
                    last_position = last_position[0] - 1
                else:
                    last_position = len(history_array)
                dict_results[metric] = score
            else:
                last_position = -1
            max_last_position = max(last_position, max_last_position)
        
        dict_results['last'] = max_last_position
        if max_last_position < parameters.get('min_iterations', 50):
            dict_results = None
            print ('not storing result from {} with iterations {}'.format(path_results, max_last_position))
        else:
            print ('storing result from {} with iterations {}'.format(path_results, max_last_position))
        
    return dict_results

## obtain_last_result_from_dict

In [ ]:
#export
def obtain_last_result_from_dict (parameters, path_results):
    name_result_file = parameters.get('result_file', 'dict_results.pk')
    path_results_file = '%s/%s' %(path_results, name_result_file)
    dict_results = None
    if os.path.exists (path_results_file):
        dict_results = pickle.load(open(path_results_file, 'rb'))
        if 'last' not in dict_results.keys() and 'epoch' in dict_results.keys():
            dict_results['last'] = dict_results['epoch']
        max_last_position = dict_results['last']
        if max_last_position < parameters.get('min_iterations', 50):
            dict_results = None
            print ('not storing result from {} with iterations {}'.format(path_results, max_last_position))
        else:
            print ('storing result from {} with iterations {}'.format(path_results, max_last_position))
    
    return dict_results